                        llama3-8b  pass@1: 0.38 | phi3 pass@1: 0.60  [HumanEval]

In [1]:
from data import CodeCompletionGenerator, load_config
from human_eval.data import write_jsonl, read_problems
from tqdm import tqdm as tqdm


config_name = "phi_3_instruct"
code_gen = CodeCompletionGenerator.from_config(config_name, device="mps")

# Get the error message on sub-batch and do fast adaptation 
system_prompt = "Do not fuck it up"
# Run Inference on HumanEval | Generate -> GetError -> SaveErro
# code_gen.run_human_eval_test(indices=indices, global_system_prompt=system_prompt)


Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

[WARNING] rope_scaling 'type' currently only supports 'linear' setting rope scaling to false.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### Fast Adaptation || Monte Carlo Tree Search || Case Study (What is an effective way of improving codeing capacity ?)

In [2]:
# Case Study Here
from src.fast_adapt import *
from src.prompt import *
import numpy as np

indices = [55] # Pin-down this Fibonacci sequence calculation issue and solve the case study
init_prompt = "Do not fuck it up"
id = 4527
node = Node.from_prompt(indices, init_prompt, id, code_gen)

Generating code completions:   0%|          | 0/2 [00:00<?, ?it/s]

Prompt: <s><|system|>
You are a friendly chatbot. [INSTRUCTION] Do not fuck it up [\INSTRUCTION] WRITE THE FULL COMPLETE FUNCTION (EG WITH def ....) END CODE WITH '```'. NOTE YOU ABSOLUTELY MUST END THE CODE WITH END CODE WITH '```' OR ELSE THE CODE WILL NOT BE INTERPRETTED!!!!<|end|>
<|user|>


def fib(n: int):
    """Return n-th Fibonacci number.
    >>> fib(10)
    55
    >>> fib(1)
    1
    >>> fib(8)
    21
    """
<|end|>
<|assistant|>


def fib(n: int):
    """Return n-th Fibonacci number.
    >>> fib(10)
    55
    >>> fib(1)
    1
    >>> fib(8)
    21
    """
    if n <= 0:
        return "Input should be positive integer."
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        a, b = 0, 1
        for _ in range(2, n):
            a, b = b, a + b
        return b

```<|end|><|assistant|> ```<|end|><|assistant|> def fib(n: int):
    """Return n-th Fibonacci number.
    >>> fib(10)
    55
    >>> fib(1)
    1
    >>> fib(8)
    21
    """
    if 

Generating code completions:  50%|█████     | 1/2 [00:07<00:07,  7.34s/it]

Prompt: 793.102 tokens-per-sec
Generation: 41.270 tokens-per-sec
Prompt: <s><|system|>
You are a friendly chatbot. [INSTRUCTION] Do not fuck it up [\INSTRUCTION] WRITE THE FULL COMPLETE FUNCTION (EG WITH def ....) END CODE WITH '```'. NOTE YOU ABSOLUTELY MUST END THE CODE WITH END CODE WITH '```' OR ELSE THE CODE WILL NOT BE INTERPRETTED!!!!<|end|>
<|user|>


def sum_to_n(n: int):
    """sum_to_n is a function that sums numbers from 1 to n.
    >>> sum_to_n(30)
    465
    >>> sum_to_n(100)
    5050
    >>> sum_to_n(5)
    15
    >>> sum_to_n(10)
    55
    >>> sum_to_n(1)
    1
    """
<|end|>
<|assistant|>


```python

def sum_to_n(n: int):
    """sum_to_n is a function that sums numbers from 1 to n.
    >>> sum_to_n(30)
    465
    >>> sum_to_n(100)
    5050
    >>> sum_to_n(5)
    15
    >>> sum_to_n(10)
    55
    >>> sum_to_n(1)
    1
    """
    return sum(range(1, n + 1))

```<|end|><|assistant|> ```python

def sum_to_n(n: int):
    """sum_to_n is a function that sums numbers f

Generating code completions: 100%|██████████| 2/2 [00:20<00:00, 10.17s/it]


Prompt: 1212.438 tokens-per-sec
Generation: 39.886 tokens-per-sec
Writing error information to data/log/phi_3_instruct-55-60_4527_info.jsonl...


100%|██████████| 2/2 [00:00<00:00, 24.28it/s]


In [11]:
# Monte Carlo Tree Search


In [10]:
child_node.prompt

'Here are the key takeaways from this error:\n\n1. **Understand the problem requirements**: Make sure you understand the problem description and what the expected output is.\n\n2. **Check the base cases**: Pay close attention to the base cases (e.g., `n == 1`, `n == 2`) as these are often where errors occur.\n\n3. **Test your code thoroughly**: Write test cases to ensure your function is working correctly.\n\n4. **Be careful with assumptions**: Make sure your assumptions about the problem and your code are correct.\n\n5. **Learn from your mistakes**: When you find the error, try to understand why it was occurring and how you can avoid similar errors in the future.\n\nIn this case, the key takeaway is to double-check the base cases and make sure they are correct. The error was caused by mistakenly returning 0 for `n == 1` and 1 for `n == 2`, which is the opposite of what it should be.\n\nBy learning from this mistake, you can avoid similar errors in the future by being more careful with

In [2]:
# Reflective Adaptation | Use Groq for fast iteration | CUDA device could also be fast (?)
from src.fast_adapt import *
from src.prompt import *
import numpy as np

batches = [indices.tolist() for indices in np.array_split(list(range(162)), 60)]
max_search = 30
pb = tqdm(total=len(batches), desc="MCTS Search Batches")
for i, indices in enumerate(batches):
    pb.desc = "MCTS Search Batches: %d/%d" % (i + 1, len(batches))
    # Initialize Sub-Batch Indices
    init_prompt = "Do not fuck it up"
    # Initialize the MCTS object
    mcts = MCTS(init_prompt, 0, indices, code_gen, max_search=max_search)

    # Run the search loop
    while not mcts.is_search_complete():
        node_to_expand = mcts.select_node()
        mcts.expand_node(node_to_expand)
        pb.update(1)
    pb.n = max_search * (i + 1)
    # Retrieve the best prompt after the search is complete
    best_prompt = mcts.get_best_prompt()
    print("Best Prompt on Batch %d: %s" % (i, best_prompt))
    mcts.save_best_node()
    # break

MCTS Search Batches:   0%|          | 0/60 [00:00<?, ?it/s]

Prompt: <s><|system|>
You are a friendly chatbot. [INSTRUCTION] Do not fuck it up [\INSTRUCTION] WRITE THE FULL COMPLETE FUNCTION (EG WITH def ....) END CODE WITH '```'. NOTE YOU ABSOLUTELY MUST END THE CODE WITH END CODE WITH '```' OR ELSE THE CODE WILL NOT BE INTERPRETTED!!!!<|end|>
<|user|>
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """
<|end|>
<|assistant|>

def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """
    for i in

Prompt: 1253.196 tokens-per-sec
Generation: 40.615 tokens-per-sec
Prompt: <s><|system|>
You are a friendly chatbot. [INSTRUCTION] Do not fuck it up [\INSTRUCTION] WRITE THE FULL COMPLETE FUNCTION (EG WITH def ....) END CODE WITH '```'. NOTE YOU ABSOLUTELY MUST END THE CODE WITH END CODE WITH '```' OR ELSE THE CODE WILL NOT BE INTERPRETTED!!!!<|end|>
<|user|>
from typing import List


def separate_paren_groups(paren_string: str) -> List[str]:
    """ Input to this function is a string containing multiple groups of nested parentheses. Your goal is to
    separate those group into separate strings and return the list of those.
    Separate groups are balanced (each open brace is properly closed) and not nested within each other
    Ignore any spaces in the input string.
    >>> separate_paren_groups('( ) (( )) (( )( ))')
    ['()', '(())', '(()())']
    """
<|end|>
<|assistant|>

from typing import List

def separate_paren_groups(paren_string: str) -> List[str]:
    # Remove spaces from t

Prompt: 1198.718 tokens-per-sec
Generation: 40.083 tokens-per-sec
Prompt: <s><|system|>
You are a friendly chatbot. [INSTRUCTION] Do not fuck it up [\INSTRUCTION] WRITE THE FULL COMPLETE FUNCTION (EG WITH def ....) END CODE WITH '```'. NOTE YOU ABSOLUTELY MUST END THE CODE WITH END CODE WITH '```' OR ELSE THE CODE WILL NOT BE INTERPRETTED!!!!<|end|>
<|user|>


def truncate_number(number: float) -> float:
    """ Given a positive floating point number, it can be decomposed into
    and integer part (largest integer smaller than given number) and decimals
    (leftover part always smaller than 1).

    Return the decimal part of the number.
    >>> truncate_number(3.5)
    0.5
    """
<|end|>
<|assistant|>


def truncate_number(number: float) -> float:
    """ Given a positive floating point number, it can be decomposed into
    and integer part (largest integer smaller than given number) and decimals
    (leftover part always smaller than 1).

    Return the decimal part of the number.


Generating code completions: 100%|██████████| 3/3 [00:27<00:00,  9.15s/it]


Prompt: 998.294 tokens-per-sec
Generation: 41.280 tokens-per-sec
Writing error information to data/log/phi_3_instruct-0-1-2_0_info.jsonl...


100%|██████████| 3/3 [00:00<00:00, 25.26it/s]


AttributeError: 'MCTS' object has no attribute 'root_node'